In [1]:
%matplotlib inline
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import random 
from tqdm import tqdm
from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomContrast,
    RandomGamma,
    RandomBrightness
)

random.seed(42)

# Important Functions

In [2]:
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 18
    
    if original_image is None and original_mask is None:
        f, ax = plt.subplots(2, 1, figsize=(8, 8))

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(8, 8))

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original image', fontsize=fontsize)
        
        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original mask', fontsize=fontsize)
        
        ax[0, 1].imshow(image)
        ax[0, 1].set_title('Transformed image', fontsize=fontsize)
        
        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed mask', fontsize=fontsize)
        
def save_aug_res(augmented_img, img_name, path_to_save, ifSoftmax=True):
    if not os.path.exists(path_to_save):
        os.mkdir(path_to_save)
    if ifSoftmax:
        
        np.save(os.path.join(path_to_save,img_name)[:-4]+'.npy', augmented_img)
    else:
        cv2.imwrite(os.path.join(path_to_save,img_name), augmented_img)

def add_back_ground(pixel):
    """
    Add forth class to the mask(I used it to add background class to all masks)
    """
    if np.count_nonzero(pixel) == 0:
        return 1
    else:
        return 0

def whole_saving_process(augmentor, train_x, train_y, x_names, path_to_save, 
                         number_of_augmentations_per_one_image=10, channels="all", ifSoftmax=True):
    """
    Save augmented images to the given path_to_save. 
    
    Args:
     - augmentator: certain type of augmentation class from albumentations library(including
                    Compose class. If smth unclear here, read albumentations lib doc.);
     - train_x: list of train images;
     - train_y: list of corresponding masks to the images;
     - x_names: list of names to the images(the names for train_x and train_y are equall);
     - path_to_save: str path, where to save the images;
     - number_of_augmentations_per_one_image: number of augmented images from 1 image in 
                                              the dataset.
     - channels: Channels has only 4 values:
                 - 'all' - save colored masks;
                 - 'a' - save binary masks(only artifact class);
                 - 'l' - save binary masks(only lumen class);
                 - 't' - save binary masks(only tissue class).
    Returns: 
     - None
                    
    """
    
    # check wether input is valid
    possible_values_of_channels = ['all', 'a', 'l', 't']
    assert channels in possible_values_of_channels, 'Input proper value of "channels"'
    
    # assign proper index value for channel corresponding to the class
    if channels == 'a':
        index = 0
    elif channels == 'l':
        index = 1
    else:
        index = 2
        
    # iterate over images in train
    for i in tqdm(range(len(train_x))):
        
        # augment one image in dataset
        for k in range(number_of_augmentations_per_one_image):
            
            if channels == 'all':
                augmented = augmentor(image=train_x[i], mask=train_y[i])
            else:
                augmented = augmentor(image=train_x[i], mask=train_y[i][:,:,index])
    
            image_my_augmented = augmented['image']
            mask_my_augmented = augmented['mask']
            
            if ifSoftmax:
                # adding background class to the mask 
                mask_my_augmented = np.dstack(
                    (
                        mask_my_augmented, 
                        np.apply_along_axis(add_back_ground, -1, mask_my_augmented)
                    )
                )
            
            save_aug_res(image_my_augmented, str(k)+'_'+x_names[i], path_to_save+'train_x', ifSoftmax=False)# here ifSoftmax should be always false!
            save_aug_res(mask_my_augmented, str(k)+'_'+x_names[i], path_to_save+'train_y', ifSoftmax)
            
    for i in range(len(train_x)):
        save_aug_res(train_x[i],x_names[i], path_to_save+'train_x', ifSoftmax=False)

        if channels == 'all':
            if ifSoftmax:
                train_y[i] = np.dstack(
                    (
                        train_y[i], 
                        np.apply_along_axis(add_back_ground, -1, train_y[i])
                    )
                )
            save_aug_res(train_y[i],x_names[i], path_to_save+'train_y', ifSoftmax)
        else:
            if ifSoftmax:
                train_y[i][:,:,index] = np.dstack(
                    (
                        train_y[i][:,:,index], 
                        np.apply_along_axis(add_back_ground, -1, train_y[i][:,:,index])
                    )
                ) 
            save_aug_res(train_y[i][:,:,index],x_names[i], path_to_save+'train_y', ifSoftmax)

def copy_softmax_masks(path_to_masks, path_to_save):
    if not os.path.exists(path_to_save):
        os.mkdir(path_to_save)
    masks_list = os.listdir(path_to_masks)
    masks = [cv2.imread(path_to_masks+masks_list[i]) for i in range(len(masks_list))]
    for i in range(len(masks)):
        proper_mask = np.dstack(
            (
                masks[i], 
                np.apply_along_axis(add_back_ground, -1, masks[i])
            )
        )
        save_aug_res(proper_mask, masks_list[i], path_to_save)


# Reading Data


In [3]:
x_path = '/home/bohdan/ultrasound/dataset/extended_dataset_preprocessed/x_train/'
# x_path = '/home/bohdan/ultrasound/dataset/augmentation/train_x/'
x_names = os.listdir(x_path)
train_x = [cv2.imread(x_path+x_names[i]) for i in range(len(x_names))]

y_path = '/home/bohdan/ultrasound/dataset/extended_dataset_preprocessed/y_train/'
# y_path = '/home/bohdan/ultrasound/dataset/augmentation/train_y/'
y_names = os.listdir(y_path)
train_y = [cv2.imread(y_path+x_names[i]) for i in range(len(x_names))]


In [ ]:
for i in range(len(train_x)):
    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.imshow(train_x[i])
    fif = np.copy(train_y[i])
    fif[fif==1] = 255
#     plt.figure(figsize=(20,10))
    plt.subplot(1,2,2)
    plt.imshow(fif)

# Creating Data Augmentators

In [4]:
full_pack_augmentor = Compose([
    
    # Non destructive transformations
#     VerticalFlip(p=0.6),
#     HorizontalFlip(p=0.6),
#     RandomRotate90(),
#     Transpose(p=0.6),
    
#     # Non-rigid transformations
    OneOf([
    ElasticTransform(p=0.5, alpha=160, sigma=180 * 0.05, alpha_affine=120 * 0.03),
    GridDistortion(p=0.5),
    OpticalDistortion(p=1, distort_limit=1, shift_limit=0.5)                  
    ], p=0.5),
    
#     Color augmentation
    RandomBrightness(p=0.5),
    RandomContrast(p=0.5),
    RandomGamma(p=0.5),
    CLAHE(p=0.5)
    ]
)

In [ ]:
augmented = full_pack_augmentor(image=train_x[0], mask=train_y[0])
image_my_augmented = augmented['image']
mask_my_augmented = augmented['mask']
mask_my_augmented[mask_my_augmented == 1] = 255
plt.figure(figsize=(10,10))
plt.imshow(mask_my_augmented)

# Augmentation Process

### RGB masks augmentation

In [7]:
PATH_TO_SAVE = '../dataset/augmentation/softmax/color_non_rigid_aug/'

whole_saving_process(full_pack_augmentor, train_x, train_y, x_names, 
                     PATH_TO_SAVE, 10, channels='all')

100%|██████████| 25/25 [06:54<00:00, 16.59s/it]


In [ ]:
PATH_TO_MASKS = '/home/bohdan/ultrasound/dataset/extended_dataset_preprocessed/y_val/'
PATH_TO_SAVE = '/home/bohdan/ultrasound/dataset/augmentation/softmax/y_val/'
copy_softmax_masks(PATH_TO_MASKS, PATH_TO_SAVE)

### Artifacts augmentation

In [ ]:
PATH_TO_SAVE = '../dataset/augmentation/softmax/artifacts_augmentation/'

whole_saving_process(full_pack_augmentor, train_x, train_y, x_names, 
                     PATH_TO_SAVE, 10, channels='a')

### Lumen augmentation

In [ ]:
PATH_TO_SAVE = '../dataset/augmentation/lumen_augmentation/'

whole_saving_process(full_pack_augmentor, train_x, train_y, x_names, 
                     PATH_TO_SAVE, 10, channels='l')

### Tissue augmentation

In [ ]:
PATH_TO_SAVE = '../dataset/augmentation/tissue_augmentation/'

whole_saving_process(full_pack_augmentor, train_x, train_y, x_names, 
                     PATH_TO_SAVE, 10, channels='t')

In [ ]:
a = 'rojfsd.png'
a[:-4]